In [1]:
import time

In [2]:
import pandas as pd

In [3]:
from hybrid_search_engine.index import build_index_from_df
from hybrid_search_engine.search_engine import SearchEngine

d:\projects\hse\hybrid-search-engine\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
meta_df = pd.read_csv("data/metadata.csv")

In [5]:
meta_df.head(5)

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,PMC,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,PMC140314,12525263.0,no-cc,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",BMC Public Health,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
1,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,PMC,Discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,PMC156578,12734001.0,no-cc,Recent analyses of human pathogens have reveal...,2003-04-28,"Disotell, Todd R",Genome Biol,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
2,le0ogx1s,NaN,PMC,A new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,PMC193621,12844350.0,no-cc,"The army of the men of death, in John Bunyan's...",2003-06-27,"Petsko, Gregory A",Genome Biol,NaN,NaN,False,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,PMC,Association of HLA class I with severe acute r...,10.1186/1471-2350-4-9,PMC212558,12969506.0,no-cc,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,"Lin, Marie; Tseng, Hsiang-Kuang; Trejaut, Jean...",BMC Med Genet,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
4,0qaoam29,5b68a553a7cbbea13472721cd1ad617d42b40c26,PMC,A double epidemic model for the SARS propagation,10.1186/1471-2334-3-19,PMC222908,12964944.0,no-cc,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,"Ng, Tuen Wai; Turinici, Gabriel; Danchin, Antoine",BMC Infect Dis,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...


In [17]:
sample_df = meta_df.head(500).copy(deep=True)
sample_df.set_index("cord_uid", inplace=True)

In [7]:

text_cols = ["title", "abstract"]
id_column = "cord_uid"

start_time = time.time()

index_df, documents_df = build_index_from_df(sample_df, columns=text_cols, id_column=id_column, filtering_columns=[])

execution_time = time.time() - start_time

print("--- %s seconds ---" % execution_time)

--- 82.6792643070221 seconds ---


In [8]:
engine = SearchEngine(index=index_df, documents_df=documents_df, columns=text_cols)

In [15]:
index_df["token vector"].apply(len)

0        200
1        200
2        200
3        200
4        200
        ... 
10759    200
10760    300
10761    300
10762    200
10763    200
Name: token vector, Length: 10764, dtype: int64

In [62]:
query = "COVID"

In [63]:
result = engine.find(query=query)

D:\Projects\HSE\Hybrid-Search-Engine\hybrid_search_engine\search_engine.py:121: RuntimeWarning: invalid value encountered in true_divide
  v = np.array([c / np.linalg.norm(c) for c in v])


In [64]:
result

,similarity
document_id,


In [47]:
sample_df.loc[result.index.values.tolist()]["abstract"][0]

'The incubation period of infectious diseases, the time from infection with a microorganism to onset of disease, is directly relevant to prevention and control. Since explicit models of the incubation period enhance our understanding of the spread of disease, previous classic studies were revisited, focusing on the modeling methods employed and paying particular attention to relatively unknown historical efforts. The earliest study on the incubation period of pandemic influenza was published in 1919, providing estimates of the incubation period of Spanish flu using the daily incidence on ships departing from several ports in Australia. Although the study explicitly dealt with an unknown time of exposure, the assumed periods of exposure, which had an equal probability of infection, were too long, and thus, likely resulted in slight underestimates of the incubation period. After the suggestion that the incubation period follows lognormal distribution, Japanese epidemiologists extended th

In [56]:
engine.nlp_engine("covid").vector

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [89]:
engine.nlp_engine("sars").vector

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [118]:
engine.nlp_engine("virus").similarity(engine.nlp_engine("virion"))

0.6269928164655674

In [66]:
from hybrid_search_engine.utils import text_processing as processing

In [69]:
processing.split_camel_case("COVID")

'COVID'

In [70]:
processing.process_string("COVID")

'covid'

In [71]:
processing.process_string("covid")

'covid'

In [76]:
index_df["token"]

False

In [90]:
import spacy

from scispacy.abbreviation import AbbreviationDetector

nlp = spacy.load("en_core_sci_lg")

In [91]:
abbradd_pipeiation_pipe = AbbreviationDetector(nlp)
nlp.add_pipe(abbreviation_pipe)

In [99]:
doc = nlp("Spinal and bulbar muscular atrophy (SBMA) is an \
           inherited motor neuron disease caused by the expansion \
           of a polyglutamine tract within the androgen receptor (AR). \
           SBMA can be caused by this easily.")

print("Abbreviation", "\t", "Definition")
for abrv in doc._.abbreviations:
    print(f"{abrv} \t ({abrv.start}, {abrv.end}) {abrv._.long_form}")

Abbreviation 	 Definition
SBMA 	 (6, 7) Spinal and bulbar muscular atrophy
SBMA 	 (33, 34) Spinal and bulbar muscular atrophy
AR 	 (29, 30) androgen receptor


In [105]:
s = "Dual Clutch Transimission (HJKLK) is a racing spec gearbox for faster gear shifting"

In [113]:
s = "Coronavirus Desease (COVID) is very infectious desease"

In [114]:
doc = nlp(s)

print("Abbreviation", "\t", "Definition")
for abrv in doc._.abbreviations:
    print(f"{abrv} \t ({abrv.start}, {abrv.end}) {abrv._.long_form}")

Abbreviation 	 Definition
COVID 	 (3, 4) Coronavirus Desease


In [102]:
doc

Dual Clutch Transimission (DCT) is a racing spec gearbox for faster gear shifting

In [94]:
doc.vector

array([ 9.47773e-02,  4.15061e-01,  4.91745e-03,  3.11639e-02,
       -1.41038e-01, -5.32763e-01,  2.74899e-01,  1.50777e-02,
        2.53135e-02,  5.00099e-01,  5.01774e-02, -4.58557e-01,
       -4.33080e-02, -4.13045e-01, -6.22302e-03,  9.64248e-02,
       -7.66348e-02, -1.60270e-01,  1.51851e-01,  5.30888e-02,
        3.99457e-01, -3.24132e-01,  2.06946e-01,  1.94619e-01,
        1.76661e-01, -1.14784e-01,  4.49287e-02, -4.49194e-04,
       -4.20595e-03, -3.48903e-01,  4.14817e-01, -3.58638e-02,
        2.40732e-01,  2.26536e-01, -1.08895e-01,  2.74919e-01,
       -5.37868e-02, -7.02954e-02,  2.03732e-01,  2.40574e-01,
       -2.81566e-01, -3.21800e-01, -3.07473e-01,  3.57916e-03,
       -8.68913e-02,  7.14537e-02, -3.38556e-01,  1.93275e-01,
       -1.53826e-01, -1.58974e-02, -1.14221e-01, -1.77224e-01,
        2.37895e-01,  3.35231e-01, -1.27043e-01, -2.96254e-03,
       -1.46683e-01, -6.80541e-02,  3.25993e-01,  1.69680e-01,
        1.84028e-01,  8.50883e-02,  1.18375e-01, -1.470

In [123]:
nlp("").vector.shape[0]

200